In [1]:
!sudo apt-get install espeak

'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip install pyttsx3


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install fuzzywuzzy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import re
from datetime import datetime
from collections import deque
import time
import os
from fuzzywuzzy import process

class MedicalDiagnosisSystem:
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.label_encoder = None
        self.max_length = 3
        self.max_words = 1000

    def initialize_system(self):
        """Initialize the medical diagnosis system"""
        print("\n🔄 Initializing Medical Diagnosis System...")
        try:
            # Load datasets
            dataset = pd.read_csv('dataset.csv')
            self.load_supplementary_data()

            # Prepare data
            symptom_cols = dataset.columns[1:]
            X = dataset[symptom_cols].fillna('').agg(' '.join, axis=1)
            y = dataset['Disease']

            # Encode labels
            self.label_encoder = LabelEncoder()
            y = self.label_encoder.fit_transform(y)

            # Prepare text data
            self.tokenizer = Tokenizer(num_words=self.max_words, oov_token="<OOV>")
            self.tokenizer.fit_on_texts(X)

            # Train model
            self.train_model(X, y)

            print("✅ System initialization complete!")
            return True

        except Exception as e:
            print(f"❌ Error during initialization: {str(e)}")
            return False

    def load_supplementary_data(self):
        """Load additional medical data"""
        self.symptom_precautions = pd.read_csv('symptom_precaution.csv')
        self.symptom_severity = pd.read_csv('Symptom-severity.csv')
        self.symptom_description = pd.read_csv('symptom_Description.csv')

    def train_model(self, X, y):
        """Train the neural network model"""
        # Convert text to sequences
        X_seq = self.tokenizer.texts_to_sequences(X)
        X_pad = pad_sequences(X_seq, maxlen=self.max_length)

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_pad, y, test_size=0.2, random_state=42
        )

        # Build model
        self.model = tf.keras.Sequential([
            tf.keras.layers.Embedding(self.max_words, 100, input_length=self.max_length),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
        ])

        # Compile and train
        self.model.compile(optimizer='adam',
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])

        self.model.fit(X_train, y_train, epochs=10, validation_split=0.2, verbose=1)

class InteractiveMedicalChatbot:
    def __init__(self, diagnosis_system):
        self.diagnosis_system = diagnosis_system
        self.conversation_history = deque(maxlen=10)
        self.current_symptoms = []
        self.last_prediction = None
        self.user_name = None
        self.available_symptoms = self.load_available_symptoms()

    def load_available_symptoms(self):
        """Load and return list of available symptoms from the dataset"""
        if hasattr(self.diagnosis_system, 'symptom_severity'):
            return self.diagnosis_system.symptom_severity['Symptom'].unique().tolist()
        return []

    def display_welcome_message(self):
        """Display welcome message and instructions"""
        os.system('cls' if os.name == 'nt' else 'clear')
        print("\n" + "="*60)
        print("🏥 Welcome to the Interactive Medical Symptom Analysis System!")
        print("="*60)

        self.user_name = input("\nPlease enter your name: ")
        print(f"\nHello {self.user_name}! 👋 I'm your medical assistant chatbot.")
        print("\nBefore we begin, please note:")
        print("1️⃣ This system is for educational purposes only")
        print("2️⃣ Always consult healthcare professionals for medical advice")
        print("3️⃣ In case of emergency, call emergency services immediately")

        input("\nPress Enter to continue...")

    def display_instructions(self):
        """Display detailed instructions"""
        print("\n📋 How to Use This System:")
        print("1. You can either describe your symptoms or choose from our list")
        print("2. The system will analyze your symptoms and suggest possible conditions")
        print("3. Follow-up questions will help refine the diagnosis")
        print("4. You can view detailed information about conditions")
        print("\n⚠️ Important: If you experience severe symptoms, seek immediate")
        print("   medical attention or call emergency services.")

        input("\nPress Enter to continue to the main menu...")

    def show_menu(self):
        """Display main menu options"""
        print("\n" + "="*40)
        print(f"🏥 Medical Assistant - {self.user_name}'s Session")
        print("="*40)
        print("\n📋 Menu Options:")
        print("1. Describe your symptoms")
        print("2. Browse symptom catalog")
        print("3. View your current symptoms")
        print("4. Emergency information")
        print("5. Help & Instructions")
        print("6. Exit")

        return input("\n👉 Please select an option (1-6): ")

    def process_symptoms(self):
        """Handle symptom input and processing"""
        print("\n🔍 Symptom Analysis")
        symptoms = []
        print("\nEnter your symptoms one by one (type 'done' when finished):")
        print("Tip: You can type part of the symptom name and we'll help you find it")
    
        while True:
            symptom_input = input("Enter symptom (or 'done' to finish): ").strip().lower()
        
            if symptom_input == 'done':
                if not symptoms:
                    print("Warning: No symptoms were entered. Are you sure you want to finish? (yes/no)")
                    if input().lower().startswith('y'):
                        break
                else:
                    print(f"\nFinal list of symptoms:")
                    for i, sym in enumerate(symptoms, 1):
                        print(f"{i}. {sym.replace('_', ' ')}")
                    break
        
            elif not symptom_input:
                print("Please enter a symptom or type 'done' to finish")
                continue
        
            symptoms.append(symptom_input)
    
        # Match entered symptoms with available symptoms using fuzzy matching
        matched_symptoms = self.match_symptoms(symptoms)
        if matched_symptoms:
            print("\n✅ Symptoms matched successfully:")
            for symptom in matched_symptoms:
                print(f"  • {symptom}")
        else:
            print("\n⚠️ No exact matches found, trying closest matches...")

        # Process symptoms and get prediction
        predictions = self.get_diagnosis(" ".join(matched_symptoms))
        self.display_diagnosis(predictions)

    def match_symptoms(self, symptoms):
        """Match user input symptoms with available symptoms using fuzzy matching"""
        symptoms_list = self.available_symptoms
        matched_symptoms = []

        for symptom in symptoms:
            # Find the closest match from the available symptoms
            closest_match = process.extractOne(symptom, symptoms_list)
            if closest_match and closest_match[1] > 70:  # If match score is above 70%
                matched_symptoms.append(closest_match[0])
            else:
                print(f"⚠️ No close match found for: {symptom}")
        return matched_symptoms

    def get_diagnosis(self, symptoms):
        """Get diagnosis based on symptoms"""
        # Clean symptoms text
        cleaned_symptoms = re.sub(r'[^a-zA-Z\s,]', '', symptoms)
        symptoms_seq = self.diagnosis_system.tokenizer.texts_to_sequences([cleaned_symptoms])
        symptoms_pad = pad_sequences(symptoms_seq,
                                   maxlen=self.diagnosis_system.max_length,
                                   padding='post')

        # Get prediction
        prediction = self.diagnosis_system.model.predict(symptoms_pad, verbose=0)

        # Get top 3 predictions
        top_indices = np.argsort(prediction[0])[-3:][::-1]
        results = []

        for idx in top_indices:
            disease = self.diagnosis_system.label_encoder.inverse_transform([idx])[0]
            confidence = prediction[0][idx]
            if confidence > 0.1:  # Minimum confidence threshold
                results.append((disease, confidence))

        return results

    def display_diagnosis(self, predictions):
        """Display diagnosis results"""
        if not predictions:
            print("\n⚠️ Unable to make a confident diagnosis based on provided symptoms.")
            print("Please provide more specific symptoms or consult a healthcare provider.")
            return

        print("\n📋 Diagnosis Results:")
        print("="*40)

        for disease, confidence in predictions:
            print(f"\n🔍 Possible Condition: {disease}")
            print(f"📊 Confidence Level: {confidence:.1%}")

            # Get disease information
            description = self.get_disease_description(disease)
            precautions = self.get_disease_precautions(disease)

            print("\n📝 Description:")
            print(description)

            if precautions:
                print("\n⚠️ Recommended Precautions:")
                for i, precaution in enumerate(precautions, 1):
                    print(f"{i}. {precaution}")

    def display_all_symptoms(self):
        """Display all available symptoms with severity levels"""
        if not self.available_symptoms:
            print("\n⚠️ No symptoms available in the database.")
            return

        print("\n📋 Available Symptoms:")
        print("="*50)

        # Group symptoms by severity if available
        if hasattr(self.diagnosis_system, 'symptom_severity'):
            severity_df = self.diagnosis_system.symptom_severity
            for severity in sorted(severity_df['weight'].unique(), reverse=True):
                symptoms = severity_df[severity_df['weight'] == severity]['Symptom'].tolist()
                print(f"\n🔵 Severity Level {severity}:")
                for symptom in sorted(symptoms):
                    print(f"  • {symptom}")
        else:
            # Simple alphabetical list if no severity data
            for symptom in sorted(self.available_symptoms):
                print(f"• {symptom}")

        input("\nPress Enter to continue...")

    def search_symptoms(self):
        """Search through available symptoms"""
        search_term = input("\nEnter search term: ").lower()

        if len(search_term) < 2:
            print("⚠️ Please enter at least 2 characters.")
            return

        matches = [s for s in self.available_symptoms
                  if search_term in s.lower()]

        if not matches:
            print("\n❌ No matching symptoms found.")
            return

        print("\n🔍 Matching Symptoms:")
        for i, symptom in enumerate(matches, 1):
            print(f"{i}. {symptom}")

        # Option to add symptoms to current list
        choice = input("\nEnter number to add symptom (or press Enter to skip): ")
        if choice.isdigit() and 0 < int(choice) <= len(matches):
            self.current_symptoms.append(matches[int(choice)-1])
            print(f"\n✅ Added '{matches[int(choice)-1]}' to your symptoms.")

    def view_symptoms_by_category(self):
        """Display symptoms grouped by body system/category"""
        # Define basic categories and their keywords
        categories = {
            "Head & Neurological": ["head", "brain", "neural", "mental"],
            "Respiratory": ["breath", "lung", "chest", "cough"],
            "Digestive": ["stomach", "digest", "bowel", "nausea"],
            "Musculoskeletal": ["muscle", "joint", "bone", "pain"],
            "Skin": ["skin", "rash", "itch"],
            "General": []  # For symptoms that don't match other categories
        }

        # Categorize symptoms
        categorized_symptoms = {cat: [] for cat in categories}

        for symptom in self.available_symptoms:
            categorized = False
            for cat, keywords in categories.items():
                if any(keyword in symptom.lower() for keyword in keywords):
                    categorized_symptoms[cat].append(symptom)
                    categorized = True
                    break
            if not categorized:
                categorized_symptoms["General"].append(symptom)

        # Display categorized symptoms
        print("\n📋 Symptoms by Category:")
        for category, symptoms in categorized_symptoms.items():
            if symptoms:
                print(f"\n🔵 {category}:")
                for symptom in sorted(symptoms):
                    print(f"  • {symptom}")

        input("\nPress Enter to continue...")

    def view_current_symptoms(self):
        """Display currently recorded symptoms"""
        if not self.current_symptoms:
            print("\n📋 No symptoms currently recorded.")
            return

        print("\n📋 Your Current Symptoms:")
        for i, symptom in enumerate(self.current_symptoms, 1):
            print(f"{i}. {symptom}")

        # Option to remove symptoms
        choice = input("\nEnter number to remove symptom (or press Enter to continue): ")
        if choice.isdigit() and 0 < int(choice) <= len(self.current_symptoms):
            removed = self.current_symptoms.pop(int(choice)-1)
            print(f"\n✅ Removed '{removed}' from your symptoms.")

    def get_disease_description(self, disease):
        """Get description for a specific disease"""
        if hasattr(self.diagnosis_system, 'symptom_description'):
            description_df = self.diagnosis_system.symptom_description
            if disease in description_df['Disease'].values:
                return description_df[description_df['Disease'] == disease]['Description'].iloc[0]
        return "Description not available."

    def get_disease_precautions(self, disease):
        """Get precautions for a specific disease"""
        if hasattr(self.diagnosis_system, 'symptom_precautions'):
            precautions_df = self.diagnosis_system.symptom_precautions
            if disease in precautions_df['Disease'].values:
                precautions = precautions_df[precautions_df['Disease'] == disease]
                return [p for p in precautions.iloc[0, 1:] if isinstance(p, str)]
        return []

    def show_emergency_info(self):
        """Display emergency information and contacts"""
        print("\n🚨 EMERGENCY INFORMATION 🚨")
        print("="*50)
        print("\nIf you experience any of these symptoms:")
        print("• Difficulty breathing or shortness of breath")
        print("• Chest pain or pressure")
        print("• Sudden severe pain")
        print("• Sudden confusion or inability to speak")
        print("• Severe allergic reaction")

        print("\n⚠️ IMMEDIATELY:")
        print("1. Call your local emergency number:")
        print("   • United States: 911")
        print("   • Europe: 112")
        print("   • UK: 999")
        print("2. Do not wait - every minute counts!")
        print("3. If possible, have someone stay with you")

        input("\nPress Enter to return to main menu...")

    def browse_symptoms(self):
        """Browse and select from available symptoms"""
        while True:
            print("\n📋 Symptom Catalog:")
            print("1. View all symptoms")
            print("2. Search symptoms")
            print("3. View by category")
            print("4. Return to main menu")

            choice = input("\nSelect an option (1-4): ")

            if choice == "1":
                self.display_all_symptoms()
            elif choice == "2":
                self.search_symptoms
                self.search_symptoms()
            elif choice == "3":
                self.view_symptoms_by_category()
            elif choice == "4":
                break

    def run(self):
        """Main chatbot loop"""
        self.display_welcome_message()
        self.display_instructions()

        while True:
            choice = self.show_menu()

            if choice == "1":
                self.process_symptoms()
            elif choice == "2":
                self.browse_symptoms()
            elif choice == "3":
                self.view_current_symptoms()
            elif choice == "4":
                self.show_emergency_info()
            elif choice == "5":
                self.display_instructions()
            elif choice == "6":
                self.display_goodbye_message()
                break
            else:
                print("\n⚠️ Invalid option. Please try again.")

    def display_goodbye_message(self):
        """Display goodbye message"""
        print("\n" + "="*60)
        print(f"👋 Thank you for using our service, {self.user_name}!")
        print("Take care and remember to consult healthcare professionals")
        print("for proper medical advice and treatment.")
        print("="*60)

# Main execution
def main():
    try:
        # Initialize the medical diagnosis system
        print("\n🏥 Starting Medical Diagnosis System...")
        diagnosis_system = MedicalDiagnosisSystem()

        if not diagnosis_system.initialize_system():
            print("❌ System initialization failed. Please check your data files.")
            return

        # Create and run the chatbot
        chatbot = InteractiveMedicalChatbot(diagnosis_system)
        chatbot.run()

    except Exception as e:
        print(f"\n❌ An error occurred: {str(e)}")
        print("Please ensure all required files are present and try again.")

if __name__ == "__main__":
    main()


🏥 Starting Medical Diagnosis System...

🔄 Initializing Medical Diagnosis System...
Epoch 1/10


/Users/lakshmimounikab/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/Users/lakshmimounikab/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


99/99 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5264 - loss: 3.2478 - val_accuracy: 0.9112 - val_loss: 0.3525
Epoch 2/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 963us/step - accuracy: 0.8936 - loss: 0.3972 - val_accuracy: 0.9201 - val_loss: 0.1482
Epoch 3/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - accuracy: 0.9192 - loss: 0.2056 - val_accuracy: 0.9289 - val_loss: 0.1250
Epoch 4/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - accuracy: 0.9255 - loss: 0.1671 - val_accuracy: 0.9213 - val_loss: 0.1271
Epoch 5/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9193 - loss: 0.1658 - val_accuracy: 0.9201 - val_loss: 0.1273
Epoch 6/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - accuracy: 0.9232 - loss: 0.1521 - val_accuracy: 0.9201 - val_loss: 0.1232
Epoch 7/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - accuracy: 0.9210 - loss: 0.1493 - val_accuracy: 0.9213 - val_loss: 0.1223
Epoch 8/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - accuracy: 0.9197 - loss: 0.1488 - val_accuracy: 0.9264 - val_loss